In [1]:
using Flight.FlightCore.Systems
using Flight.FlightCore.Plotting

using Flight.FlightPhysics.Geodesy
using Flight.FlightPhysics.Kinematics
using Flight.FlightComponents.Aircraft
using Flight.FlightComponents.Control
using Flight.FlightAircraft.C172R
using Flight.FlightAircraft.C172

using UnPack
using ControlSystems
using RobustAndOptimalControl
using ComponentArrays
using LinearAlgebra

In [4]:
ac = Cessna172RBase(NED()) |> System #linearization requires NED kinematics

#same mass in all cases
fwd_cg_pld = C172.PayloadU(m_pilot = 100, m_copilot = 100, m_baggage = 0)
aft_cg_pld = C172.PayloadU(m_pilot = 50, m_copilot = 50, m_baggage = 100)
mid_cg_pld = C172.PayloadU(m_pilot = 75, m_copilot = 75, m_baggage = 50)

design_condition = C172.TrimParameters(
    Ob = Geographic(LatLon(), HOrth(1000)),
    EAS = 40.0,
    γ_wOb_n = 0.0,
    x_fuel = 0.5,
    flaps = 0.0,
    payload = mid_cg_pld)

lss_lon = Control.LinearStateSpace(ac, design_condition; model = :lon);

# nss_lon = named_ss(lss_lon);
# nss_pitch = named_ss(lss_pitch);
# lss_pitch.A[:θ, :]

LinearStateSpace{5, 1, 4, ComponentVector{Float64, Vector{Float64}, Tuple{Axis{(q = 1, v_x = 2, v_z = 3, α_filt = 4, θ = 5)}}}, ComponentVector{Float64, Vector{Float64}, Tuple{Axis{(elevator = 1,)}}}, ComponentVector{Float64, Vector{Float64}, Tuple{Axis{(q = 1, θ = 2, α = 3, EAS = 4)}}}, ComponentMatrix{Float64, Matrix{Float64}, Tuple{Axis{(q = 1, v_x = 2, v_z = 3, α_filt = 4, θ = 5)}, Axis{(q = 1, v_x = 2, v_z = 3, α_filt = 4, θ = 5)}}}, ComponentMatrix{Float64, Matrix{Float64}, Tuple{Axis{(q = 1, v_x = 2, v_z = 3, α_filt = 4, θ = 5)}, Axis{(elevator = 1,)}}}, ComponentMatrix{Float64, Matrix{Float64}, Tuple{Axis{(q = 1, θ = 2, α = 3, EAS = 4)}, Axis{(q = 1, v_x = 2, v_z = 3, α_filt = 4, θ = 5)}}}, ComponentMatrix{Float64, Matrix{Float64}, Tuple{Axis{(q = 1, θ = 2, α = 3, EAS = 4)}, Axis{(elevator = 1,)}}}}((q = -8.525268370775242e-9, v_x = -1.8306764676097415e-6, v_z = -6.12739779654382e-7, α_filt = 0.0, θ = 0.0), (q = -6.626666458540626e-6, v_x = 41.880598801593656, v_z = 3.024085505

Let's start with a SIMO model describing pitch dynamics. We will only consider the elevator as a control input and drop the altitude and engine speed states. For simplicity, we will also drop the filtered AoA state. Although it has a significant effect on the short period mode, it doesn't change the structure of the problem.

In [52]:
# x_labels_pitch = [:q, :v_x, :v_z, :α_filt, :θ]
x_labels_pitch = [:q, :v_x, :v_z, :θ]
u_labels_pitch = [:elevator]
y_labels_pitch = [:q, :θ, :α, :EAS]
lss_pitch = submodel(lss_lon; x = x_labels_pitch, u = u_labels_pitch, y = y_labels_pitch)
nss_pitch = named_ss(lss_pitch)
dampreport(nss_pitch)

|        Pole        |   Damping     |   Frequency   |   Frequency   | Time Constant |
|                    |    Ratio      |   (rad/sec)   |     (Hz)      |     (sec)     |
+--------------------+---------------+---------------+---------------+---------------+
| -0.0436 ±  0.312im |  0.138        |  0.315        |  0.0502       |  23           |
| -3.44   ±   12.3im |  0.27         |  12.8         |  2.03         |  0.29         |


In [98]:
controllability(nss_pitch)

(iscontrollable = true, ranks = [4, 4, 4, 4], sigma_min = [7.322654941169901, 7.322654941169901, 0.03085565865791449, 0.03085565865791449])

Following Stengel's notation from section 6.2, here, $n=4$, $m=1$ and:

In [53]:
F = copy(nss_pitch.A)
G = copy(nss_pitch.B)
display(F)
display(G)

4×4 Matrix{Float64}:
 -4.09388    0.250311    -3.69306     -3.65845e-6
 -0.476628  -0.225462     2.43869     -9.7518
 40.7319    -0.274284    -2.65489     -0.70414
  0.999999   1.57407e-7   1.13659e-8  -2.23012e-14

4×1 Matrix{Float64}:
  9.427519146233827
 -6.385995985702365
  3.136085390848322
  0.0

The last row of F indicates that $\theta$ is an exact integral of $q$. Were it not for numerical error and curved Earth effects, this row would likely be [1, 0, 0, 0]. So we set:

In [70]:
F[end, :] .= [1, 0, 0, 0]
display(F)
cond(F)

4×4 Matrix{Float64}:
 -4.09388    0.250311  -3.69306  -3.65845e-6
 -0.476628  -0.225462   2.43869  -9.7518
 40.7319    -0.274284  -2.65489  -0.70414
  1.0        0.0        0.0       0.0

4011.831663090734

We see that $F$ is not particularly ill-conditioned, so there are no invertibility issues. This makes sense. There are no redundant or fully decoupled states.

Now we need to define the matrix blocks that select the command vector. In this case, the command vector is simply $q$, so:

In [97]:
H_x = nss_pitch.C[1, :]' |> collect
H_u = 0.0
display(H_x)

1×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0

With, $F$, $G$, $H_x$ and $H_u$ we construct matrix $A$:

In [72]:
A = [F G; H_x H_u]
display(A)
cond(A)

5×5 Matrix{Float64}:
 -4.09388    0.250311  -3.69306  -3.65845e-6   9.42752
 -0.476628  -0.225462   2.43869  -9.7518      -6.386
 40.7319    -0.274284  -2.65489  -0.70414      3.13609
  1.0        0.0        0.0       0.0          0.0
  1.0        0.0        0.0       0.0          0.0

5.335726251339051e34

As expected, $A$ is singular. Since $F$ is not, this must be because the term $-H_x F^{-1}G + H_u$ is singular. Indeed:

In [73]:
display(-H_x * inv(F) * G .+ H_u)

1×1 Matrix{Float64}:
 0.0

Thus, we have to work with a quasi-static equilibrium. We partition the state vector in two blocks, one of size $n_1=3$ containing $q$, $v_x$ and $v_y$, and the other of size $n_2=1$, containing $\theta$. Then:

In [101]:
n_1 = 3
n_2 = 1
F_11 = F[1:n_1, 1:n_1]
F_12 = F[1:n_1, n_1+1:end]
F_21 = F[n_1+1:end, 1:n_1]
F_22 = F[n_1+1:end, n_1+1:end]
G_1 = G[1:n_1, :]
G_2 = G[n_1+1, :]
H_x1 = H_x[:, 1:n_1]
H_x2 = H_x[:, n_1+1]

display(F_11)
display(F_12)
display(F_21)
display(F_22)

3×3 Matrix{Float64}:
 -4.09388    0.250311  -3.69306
 -0.476628  -0.225462   2.43869
 40.7319    -0.274284  -2.65489

3×1 Matrix{Float64}:
 -3.6584495396452255e-6
 -9.751795990188938
 -0.7041396945986307

1×3 Matrix{Float64}:
 1.0  0.0  0.0

1×1 Matrix{Float64}:
 0.0

Now we construct the matrix:

In [102]:
A_red = [F_11 G_1; H_x1 H_u]
B = inv(A_red)
B_11 = B[1:n_1, 1:n_1]
B_12 = B[1:n_1, n_1+1:end]
B_21 = B[n_1+1:end, 1:n_1]
B_22 = B[n_1+1:end, n_1+1:end]
display(B)


4×4 Matrix{Float64}:
   0.0        0.0       0.0          1.0
 -17.2416   -24.9138    1.09876   -127.214
   4.55512    6.24508  -0.976494    61.3992
   2.34824    3.10789  -0.411697    27.864